In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as AerSampler   
from qiskit.circuit.library import RealAmplitudes
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.algorithms import QSVC


In [2]:
df = pd.read_csv('../stunting_dataset_10k.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Jenis Kelamin      10000 non-null  int64  
 1   Umur (bulan)       10000 non-null  int64  
 2   Tinggi Badan (cm)  10000 non-null  float64
 3   Berat Badan (kg)   10000 non-null  float64
 4   Stunting           10000 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 390.8 KB


In [3]:
X = df.drop('Stunting', axis=1)
y = df['Stunting']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

# --- 1. Reduksi dimensi ---
pca = PCA(n_components=0.95, svd_solver='full')   # jaga ≥95 % varians
X_train_red = pca.fit_transform(X_train_scaled)
X_test_red  = pca.transform(X_test_scaled)
num_qubits  = X_train_red.shape[1]

# --- 2. Quantum kernel ---
feature_map = RealAmplitudes(num_qubits=num_qubits, reps=1)
sampler  = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
q_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

# --- 3. SVM training ---
model = SVC(kernel=q_kernel.evaluate, C=1.0)
model.fit(X_train_red, y_train)

y_pred_train = model.predict(X_train_red)
y_pred_test   = model.predict(X_test_red)

print('train')
acc = accuracy_score(y_train, y_pred_train)
print(f"Akurasi train QSVM  : {acc:.3f}")
print("\n=== Classification Report ===")
print(classification_report(y_train, y_pred_train))

print('test')
acc = accuracy_score(y_test, y_pred_test)
print(f"Akurasi test QSVM  : {acc:.3f}")
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred_test))


MemoryError: 